### Prompt

A retail chain has 25 stores carrying variety of items.  Not every store carries the same items and each has its own level of prioritization within the chain and different required stock levels.  There is a central warehouse that contains all of the available items.

The Goal:
Distribute items from the warehouse to each store, filling the available stock at each store in order of the store's priority.

In [84]:
# importing lib

import pandas as pd
import re
import os
import numpy as np

In [85]:
# importing data

cwd = os.getcwd()

file_names = ['data.csv', 'lookup.csv', 'items.csv']

file_path = []

for x in file_names:
    
    file_path.append(os.path.join(cwd, x))
    
dfs={}

for x in file_path:
    
    name = os.path.basename(x)
    
    dfs[name] = pd.read_csv(x)

df_data =  dfs['data.csv']

df_lookup = dfs['lookup.csv']

df_items = dfs['items.csv']

In [86]:
df_data.head()

,Store,Priority,City,State,Lon,Lat
0,A,26,Houston,TX,-95.562207,29.839187
1,B,4,Birmingham,AL,-86.625591,33.410321
2,C,25,Seattle,WA,-122.291399,47.603993
3,D,3,Detroit,MI,-83.255088,42.412144
4,E,5,Nashville,TN,-86.729174,36.135464


In [87]:
df_lookup.head()

,Store,Item,Required
0,A,1,47
1,A,2,29
2,A,3,50
3,A,4,12
4,A,5,19


In [88]:
df_items.head()

,Warehouse,Item,Count
0,Main,1,824
1,Main,2,1145
2,Main,3,1354
3,Main,4,916
4,Main,5,1120


In [89]:
# joining data and lookup 

df_merge = df_data.merge(df_lookup, on = 'Store')

df_merge.head()

,Store,Priority,City,State,Lon,Lat,Item,Required
0,A,26,Houston,TX,-95.562207,29.839187,1,47
1,A,26,Houston,TX,-95.562207,29.839187,2,29
2,A,26,Houston,TX,-95.562207,29.839187,3,50
3,A,26,Houston,TX,-95.562207,29.839187,4,12
4,A,26,Houston,TX,-95.562207,29.839187,5,19


In [90]:
# sorting on item and priority

df_merge.sort_values(by = ['Item', 'Priority'], inplace = True)

df_merge.head()

,Store,Priority,City,State,Lon,Lat,Item,Required
56,H,1,Fayetteville,NC,-78.907738,35.024302,1,11
37,F,2,Chicago,IL,-87.597287,41.734000,1,71
30,E,5,Nashville,TN,-86.729174,36.135464,1,44
89,L,6,Los Angeles,CA,-118.386826,33.945722,1,26
114,O,8,Dallas,TX,-96.770293,32.846730,1,15


### Building and Testing Iterative Function

In [116]:
def item_distribution(df, df_items):
    
    distribution_list = []
                                       
    for index, row in df.iterrows():
        
        # merging on item with items data
        
        merged = row.to_frame().T.merge(df_items, on = 'Item')
        
        # calculating items assigned to store
        
        i_available = merged['Count']
        i_required = merged['Required']
        
        condition = [(i_available >= i_required),
                     (i_available < i_required) & (i_available > 0),
                     (i_available < 0)]
        
        choices = [i_required, i_available, 0]
        
        merged['assigned'] = np.select(condition, choices)
        
        # recalculating available items
        
        merged['Count'] = merged['Count'] - merged['Required']
        
        # appending merged to list
        
        distribution_list.append(merged)
        
        # unioning merged to rest of df_items
        
        unmerged = df_items.merge(merged, on = 'Item', how = 'left') # left joining
        
        unmerged = unmerged[unmerged['Store'].isna()] # filtering out where store is populated (to return items that didn't join)
        
        unmerged = unmerged.iloc[:, 0:3]
        
        unmerged.columns = ['Warehouse', 'Item', 'Count']
        
        merged = merged[['Warehouse', 'Item', 'Count']]
        
        df_items = pd.concat([merged, unmerged])
    
    return pd.concat(distribution_list)
        

In [118]:
# applying function

df_solution = item_distribution(df_merge, df_items)

df_solution.head()

,Store,Priority,City,State,Lon,Lat,Item,Required,Warehouse,Count,assigned
0,H,1,Fayetteville,NC,-78.907738,35.024302,1,11,Main,813,11
0,F,2,Chicago,IL,-87.597287,41.734,1,71,Main,742,71
0,E,5,Nashville,TN,-86.729174,36.135464,1,44,Main,698,44
0,L,6,Los Angeles,CA,-118.386826,33.945722,1,26,Main,672,26
0,O,8,Dallas,TX,-96.770293,32.84673,1,15,Main,657,15


In [119]:
df_solution[df_solution['assigned'] == 0] # checking that appropriate warehouses were assigned 0 items

,Store,Priority,City,State,Lon,Lat,Item,Required,Warehouse,Count,assigned
0,A,26,Houston,TX,-95.562207,29.839187,6,72,Main,-73,0
0,A,26,Houston,TX,-95.562207,29.839187,8,2,Main,-15,0
0,I,23,Miami,FL,-80.31047,25.792304,9,16,Main,-50,0
0,G,24,Lansing,IA,-91.228717,43.361018,9,30,Main,-80,0
